In [16]:
# Importing all dependancies

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [17]:
# Define credentials for the foursquare account
CLIENT_ID = 'SDP01TDTGMVFVVDTBTFBJET4FUEGGZLW2BV312BKIQ1WMQLW' # your Foursquare ID
CLIENT_SECRET = 'XS314VI5I2KKNI3QUQXJ352WG2ISSPJ4WBK3VCGZX0FT4RMQ' # your Foursquare Secret
ACCESS_TOKEN = 'CJVKEFTG0ADN5F30LK0LFV1ZIFTOKGVROFCF2ZXZZJHE2VQY' # your FourSquare Access Token
VERSION = '20180604'

In [18]:
# Define everything to find restaurants in a specific city

# Link to list of the postal codes
file_name = 'https://raw.githubusercontent.com/pzahlen/Coursera_Capstone/master/Postal_Code_Bruxelles.csv'
addresses = pd.read_csv(file_name)
# Category of restaurants used for this example is french restaurants
search_query = 'French Restaurant'
# Search radius is 1000 m
radius = 1000
# Maximal limit of received addresses back from foursquare
LIMIT = 50

In [19]:
# Creating an example dataframe of the received data
address = '1000 Bruxelles-Ville'
geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
df = json_normalize(venues)
# Create empty shell for adding data later
df[0:0]

# Iterate through the different postal codes and put everything to the df dataframe
for i in addresses.index:
    geolocator = Nominatim(user_agent = "foursquare_agent")
    location = geolocator.geocode(addresses['PostalCode'][i])
    latitude = location.latitude
    longitude = location.longitude
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    temp = json_normalize(venues)
    df = pd.concat([df, temp], ignore_index = True)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [20]:
# Previewing the collected data
df.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,509d18bde4b07eb4d8229833,Frenchkick,"[{'id': '4bf58dd8d48988d104951735', 'name': 'B...",v-1613813228,False,Rue Marche Au Charbon 11,11,50.846585,4.350772,"[{'label': 'display', 'lat': 50.846585, 'lng':...",65,1000,BE,Brussel,Brussels Hoofdstedelijk Gewest,België,"[Rue Marche Au Charbon 11 (11), 1000 Brussel]",NaN,NaN
1,4eb2fbb8b80328580a017302,Brasserie de l'Ommegang,"[{'id': '57558b36e4b065ecebd306b0', 'name': 'B...",v-1613813228,False,Grote Markt 9 Grand Place,NaN,50.846566,4.352609,"[{'label': 'display', 'lat': 50.84656550922511...",64,1000,BE,Brussel,Brussels Hoofdstedelijk Gewest,België,"[Grote Markt 9 Grand Place, 1000 Brussel]",NaN,NaN
2,50b5f09be4b0f5cb08770674,Restaurant NMBS / SNCB,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1613813228,False,Europakruispunt 2 Carrefour de l'Europe,NaN,50.846345,4.357042,"[{'label': 'display', 'lat': 50.84634510447499...",376,1000,BE,Brussel,Brussels Hoofdstedelijk Gewest,België,"[Europakruispunt 2 Carrefour de l'Europe, 1000...",NaN,NaN
3,51d96c40498ec79de3eea97d,Restaurant Poissons Grill,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1613813228,False,Rue de Brabant 67,NaN,50.846563,4.353374,"[{'label': 'display', 'lat': 50.84656277189522...",117,NaN,BE,Brusseld,NaN,België,"[Rue de Brabant 67, Brusseld]",NaN,NaN
4,5cc57f751acf11002c9bc2cc,the blue restaurant,"[{'id': '52e81612bcbc57f1066b7a02', 'name': 'B...",v-1613813228,False,NaN,NaN,50.845667,4.353267,"[{'label': 'display', 'lat': 50.845667, 'lng':...",148,NaN,BE,Brussel,Brussels Hoofdstedelijk Gewest,België,[Brussel],NaN,NaN


In [21]:
# Cleaning of the data

# Just using the necessary columns
df = df.filter(['id', 'name', 'location.address', 'location.lat', 'location.lng', 'location.postalCode'])

# Renaming certain columns
df = df.rename(columns={'location.address': 'address', 'location.lat': 'latitude', 'location.lng': 'longitude', 'location.postalCode': 'postal code'})

# Dropping duplicates
df = df.drop_duplicates()

# Dropping the rows where we have no values
df = df.dropna()

In [22]:
# Adding a seventh column with the name 'rating'
df.insert(6, 'rating', '')

In [23]:
df.head()

,id,name,address,latitude,longitude,postal code,rating
0,509d18bde4b07eb4d8229833,Frenchkick,Rue Marche Au Charbon 11,50.846585,4.350772,1000,
1,4eb2fbb8b80328580a017302,Brasserie de l'Ommegang,Grote Markt 9 Grand Place,50.846566,4.352609,1000,
2,50b5f09be4b0f5cb08770674,Restaurant NMBS / SNCB,Europakruispunt 2 Carrefour de l'Europe,50.846345,4.357042,1000,
5,52e2df4c498ed7af43f58713,Rustic - Restaurant romanesc,Rue Gretry,50.849873,4.350758,1000,
6,4c7cd7b4744d8cfadfa2bb37,French Touch,Marnixlaan 17 avenue Marnix,50.839198,4.363390,1000,


In [9]:
# Using a for loop to get the rating based on the id
for venue_id in df['id']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        print(json_normalize(result['response']['venue']['rating']))
    except:
# This method is not viable because it is not possible to get the ratings from foursquare api as getting the rating is a premium service

SyntaxError: unexpected EOF while parsing (<ipython-input-9-9c915d35e4ba>, line 8)

In [24]:
# The solution is to use random rating numbers
data = np.random.randint(0, 10, size = len(df))
df_random = pd.DataFrame(data, columns=['rating'])
df['rating'] = df_random

In [25]:
df.head()

,id,name,address,latitude,longitude,postal code,rating
0,509d18bde4b07eb4d8229833,Frenchkick,Rue Marche Au Charbon 11,50.846585,4.350772,1000,8.0
1,4eb2fbb8b80328580a017302,Brasserie de l'Ommegang,Grote Markt 9 Grand Place,50.846566,4.352609,1000,5.0
2,50b5f09be4b0f5cb08770674,Restaurant NMBS / SNCB,Europakruispunt 2 Carrefour de l'Europe,50.846345,4.357042,1000,0.0
5,52e2df4c498ed7af43f58713,Rustic - Restaurant romanesc,Rue Gretry,50.849873,4.350758,1000,0.0
6,4c7cd7b4744d8cfadfa2bb37,French Touch,Marnixlaan 17 avenue Marnix,50.839198,4.363390,1000,6.0


In [28]:
# Kmeans clustering
k = len(addresses)
bruxelles_clustering = df.drop(['id','name','address', 'rating'],1)
kmeans = KMeans(n_clusters = k, random_state = 0).fit(bruxelles_clustering)
kmeans.labels_
df.insert(0, 'Cluster Labels', kmeans.labels_)

In [31]:
df.head()

,Cluster Labels,id,name,address,latitude,longitude,postal code,rating
0,0,509d18bde4b07eb4d8229833,Frenchkick,Rue Marche Au Charbon 11,50.846585,4.350772,1000,8.0
1,0,4eb2fbb8b80328580a017302,Brasserie de l'Ommegang,Grote Markt 9 Grand Place,50.846566,4.352609,1000,5.0
2,0,50b5f09be4b0f5cb08770674,Restaurant NMBS / SNCB,Europakruispunt 2 Carrefour de l'Europe,50.846345,4.357042,1000,0.0
5,0,52e2df4c498ed7af43f58713,Rustic - Restaurant romanesc,Rue Gretry,50.849873,4.350758,1000,0.0
6,0,4c7cd7b4744d8cfadfa2bb37,French Touch,Marnixlaan 17 avenue Marnix,50.839198,4.363390,1000,6.0


In [33]:
# create map
map_clusters = folium.Map(location = [50.850340, 4.351710], zoom_start = 12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df['latitude'], df['longitude'], df['postal code'], df['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now we could continue to tweak even more and to solve the problem graphicaly but there is much more easier solution for our problem.
Beneath I will just use some simple group mathematics to solve our problem.

Finding the best place by simply grouping by postal code and taking the mean.
Here we use the power of pandas to do that very easily.

In [36]:
# Simple grouping and mean calculation
grouped = df.groupby('postal code').agg({'rating': ['mean']})
grouped

,rating
,mean
postal code,
1000,4.617647
1009,NaN
1030,3.750000
1040,6.000000
1050,1.666667
1060,NaN
1070,NaN
1080,0.000000


In [37]:
# Sometimes because the lack of data, some postal codes gives us no information
# Dropping the empty rows
grouped_end = grouped.dropna()
grouped_end

,rating
,mean
postal code,
1000,4.617647
1030,3.750000
1040,6.000000
1050,1.666667
1080,0.000000
7000,8.000000


Based on this the best place to open a restaurant is the postal code with the lowest value.